In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.learner import *

import torchtext
from torchtext import vocab, data
from torchtext.datasets import language_modeling

from fastai.rnn_reg import *
from fastai.rnn_train import *
from fastai.nlp import *
from fastai.lm_rnn import *

import dill as pickle
import spacy
import os
import glob
import pandas as pd

In [10]:
ls

fastai@                                             test.tsv
movie-review-sentiment-analysis-kernels-only.ipynb  train.tsv
sampleSubmission.csv


In [30]:
df_train = pd.read_csv("train.tsv", sep="\t")

In [3]:
df_test = pd.read_csv("test.tsv",sep="\t")

In [4]:
df_test.head()

,PhraseId,SentenceId,Phrase
0,156061,8545,An intermittently pleasing but mostly routine ...
1,156062,8545,An intermittently pleasing but mostly routine ...
2,156063,8545,An
3,156064,8545,intermittently pleasing but mostly routine effort
4,156065,8545,intermittently pleasing but mostly routine


In [7]:
df_train.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [31]:
#drop incomplete sentences in training set
sentences = []
for i in df_train["SentenceId"].unique():  
    sentences.append([df_train[df_train["SentenceId"] == i].reset_index().loc[0]["Phrase"],
                      df_train[df_train["SentenceId"] == i].reset_index().loc[0]["Sentiment"]])
#     print(df[i].reset_index().loc[0]["Phrase"])
df_train = pd.DataFrame(sentences)

In [32]:
len(df_train)

8529

In [35]:
count = 1
validRatio = 0.3
L = len(df_train)
cutPoint = int(round((1-validRatio)*L))

for index, row in df_train.iterrows():
    phrase=row[0]
    label = row[1]
    if (count<cutPoint):
        filename = os.path.join("labeled_text","train", str(label), str(count))+".txt"
    else:
        filename = os.path.join("labeled_text","valid", str(label), str(count))+".txt"
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    with open(filename,"w") as f:
        f.write(phrase)
    count = count + 1

In [36]:
#total number of sentences
count

8530

In [40]:
#of examples in training set
cutPoint-1

5969

In [38]:
#Show labels
labelList = !ls labeled_text/train
labelList

['0', '1', '2', '3', '4']

In [42]:
#Count number of samples under each label
labelCount = []
for i in labelList:
    trn_files = !ls labeled_text/train/{i}
    labelCount.append(len(trn_files))
labelCount

[755, 1550, 1153, 1620, 891]

In [62]:
#Review sentences
label = 3
fileIndex = 40
FileList = !ls labeled_text/train/{label}
sample = !cat labeled_text/train/{label}/{FileList[fileIndex]}
sample

['It throws quirky characters , odd situations , and off-kilter dialogue at us , all as if to say , `` Look at this !']

In [53]:
class MovieReview(torchtext.data.Dataset):
    """Create a MovieReview dataset instance given a path and fields.
    Arguments:
        path: Path to the dataset's highest level directory
        text_field: The field that will be used for text data.
        label_field: The field that will be used for label data.
        Remaining keyword arguments: Passed to the constructor of
            data.Dataset.
    """
    def __init__(self, path, text_field, label_field, **kwargs):
        fields = [('text', text_field), ('label', label_field)]
        examples = []
        for label in ['0','1','2','3','4']:
            fnames = glob.glob(os.path.join(path, label, "*.txt"))
            assert fnames, f"can't find file under {path}/{label}"
            for fname in fnames:
                with open(fname, "r") as f: text = f.readline()
                examples.append(data.Example.fromlist([text,label], fields))
        super().__init__(examples, fields, **kwargs)
    
    @classmethod
    def splits(cls, text_field, label_field, root='.data', 
               train='train', test='test', **kwargs):
        return super().splits(
            root, text_field=text_field, label_field=label_field,
            train=train, validation=None, test=test, **kwargs)
    
        

In [54]:
MovieReviewLabel = data.Field(sequential=False)

In [55]:
TEXT = pickle.load(open("TEXT.pkl","rb")) #open in read and bineary mode

In [56]:
PATH = "labeled_text"

In [57]:
splits = MovieReview.splits(TEXT, MovieReviewLabel, PATH,train="train", test="valid")

In [71]:
t = splits[0].examples[0]

In [72]:
t.label, " ".join(t.text[:16])

('0',
 'the script was reportedly rewritten a dozen times -- either 11 times too many or else')

In [81]:
bs = 64; bptt = 70

In [74]:
md2 = TextData.from_splits(PATH, splits, bs)

In [75]:
ls

adam3_10_enc-Copy1.h5                               sampleSubmission.csv
fastai@                                             test.tsv
labeled_text/                                       TEXT.pkl
movie-review-sentiment-analysis-kernels-only.ipynb  train.tsv


In [77]:
opt_fn = partial(optim.Adam, betas=(0.7, 0.99))

In [83]:
em_sz = 200  # size of each embedding vector
nh = 500     # number of hidden activations per layer
nl = 3       # number of layers

In [85]:
m3 = md2.get_model(opt_fn, 1500, bptt, emb_sz=em_sz, n_hid=nh, n_layers=nl, 
           dropout=0.1, dropouti=0.4, wdrop=0.5, dropoute=0.05, dropouth=0.3)
m3.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
m3.load_encoder(f'adam3_10_enc')

In [86]:
m3.clip=25.
lrs=np.array([1e-4,1e-4,1e-4,1e-3,1e-2])

In [87]:
m3.fit(lrs, 7, metrics=[accuracy], cycle_len=2, cycle_save_name='MovieREview')

  0%|          | 0/94 [00:00<?, ?it/s]


TypeError: '<' not supported between instances of 'Example' and 'Example'